## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset ##

In [2]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv("abcnews-date-text.csv");
# We only need the Headlines text column from the data
data_text = data[['headline_text']];

In [3]:
'''
Add an index column to the dataset and save the dataset as 'documents'
'''
data_text['index'] = data_text.index
documents = data_text
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
5,ambitious olsson wins triple jump,5
6,antic delighted with record breaking barca,6
7,aussie qualifier stosur wastes four memphis match,7
8,aust addresses un security council over iraq,8
9,australia is locked into war timetable opp,9


In [4]:
'''
Get the total number of documents
'''
print(len(documents))

1103665


In [5]:
'''
Preview a document and assign the index value to 'document_num'
'''
document_num = 4310
print("\n**Printing out a sample document:**")
print(documents[documents['index'] == document_num])


**Printing out a sample document:**
                    headline_text  index
4310  rain helps dampen bushfires   4310


In [6]:
'''
Seperate the value of the headline from the document selected with 'document_num'
'''
# TODO
print(documents[documents['index'] == document_num].values[0][0])

rain helps dampen bushfires


** Looks like this document will come under `Politics`. **

## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [7]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:
'''
Lemmatizing example for a verb, noun.
'''
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


In [10]:
'''
Stemming example
'''
stemmer = SnowballStemmer("english")
plurals = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in plurals]
print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [11]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result = [lemmatize_stemming(word) for word in simple_preprocess(text) if word not in STOPWORDS and len(word) > 3]
    return result

In [12]:
'''
Preview a document after preprocessing
'''

doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [14]:
'''
Save the values of the headlines into a variable 'training_headlines' from 'documents'
'''
training_headlines = documents['headline_text']

In [15]:
'''
Preview 'training_headlines'
'''
training_headlines

0          aba decides against community broadcasting lic...
1             act fire witnesses must be aware of defamation
2             a g calls for infrastructure protection summit
3                   air nz staff in aust strike for pay rise
4              air nz strike to affect australian travellers
5                          ambitious olsson wins triple jump
6                 antic delighted with record breaking barca
7          aussie qualifier stosur wastes four memphis match
8               aust addresses un security council over iraq
9                 australia is locked into war timetable opp
10         australia to contribute 10 million in aid to iraq
11         barca take record as robson celebrates birthda...
12                                bathhouse plans move ahead
13             big hopes for launceston cycling championship
14                    big plan to boost paroo water supplies
15                    blizzard buries united states in bills
16            brigadier 

In [16]:
'''
Perform preprocessing on entire dataset using the function defined earlier and save it to 'processed_docs'
'''
processed_docs = training_headlines.apply(lambda doc: preprocess(doc))

In [17]:
'''
Preview 'processed_docs'
'''
processed_docs

0                      [decid, communiti, broadcast, licenc]
1                                         [wit, awar, defam]
2                     [call, infrastructur, protect, summit]
3                                [staff, aust, strike, rise]
4                       [strike, affect, australian, travel]
5                         [ambiti, olsson, win, tripl, jump]
6                     [antic, delight, record, break, barca]
7              [aussi, qualifi, stosur, wast, memphi, match]
8                      [aust, address, secur, council, iraq]
9                                   [australia, lock, timet]
10                     [australia, contribut, million, iraq]
11                 [barca, record, robson, celebr, birthday]
12                                   [bathhous, plan, ahead]
13                     [hop, launceston, cycl, championship]
14                       [plan, boost, paroo, water, suppli]
15                       [blizzard, buri, unit, state, bill]
16                 [brig

## Step 3.1: Bag of words on the dataset ##

In [18]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set using gensim.corpora.Dictionary
and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [19]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [20]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1)

** Gensim doc2bow **

`doc2bow(document)`

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [21]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc)  for doc in processed_docs ]

In [22]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
# document_num = 4310
bow_corpus[document_num]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [23]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [27]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [28]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf =tfidf[bow_corpus]

In [29]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.58929086447099832),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.50465203286956617)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [33]:
# LDA mono-core
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 10, id2word = dictionary, passes = 1)

In [34]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.024*"perth" + 0.021*"brisban" + 0.014*"show" + 0.014*"polit" + 0.013*"feder" + 0.011*"record" + 0.009*"weather" + 0.008*"time" + 0.008*"year" + 0.007*"farmer"


Topic: 1 Word: 0.025*"north" + 0.022*"canberra" + 0.020*"coast" + 0.016*"say" + 0.015*"labor" + 0.014*"china" + 0.013*"korea" + 0.012*"gold" + 0.012*"senat" + 0.011*"vote"


Topic: 2 Word: 0.057*"australia" + 0.025*"world" + 0.018*"open" + 0.016*"final" + 0.013*"win" + 0.012*"leagu" + 0.012*"share" + 0.012*"take" + 0.009*"lead" + 0.009*"young"


Topic: 3 Word: 0.050*"polic" + 0.024*"death" + 0.019*"charg" + 0.016*"live" + 0.016*"shoot" + 0.014*"murder" + 0.014*"woman" + 0.012*"fight" + 0.012*"investig" + 0.011*"child"


Topic: 4 Word: 0.025*"govern" + 0.017*"donald" + 0.017*"plan" + 0.016*"chang" + 0.012*"health" + 0.012*"indigen" + 0.011*"council" + 0.011*"work" + 0.011*"fund" + 0.010*"say"


Topic: 5 Word: 0.025*"melbourn" + 0.024*"nation" + 0.014*"west" + 0.013*"state" + 0.013*"bank" + 0.013*"street" + 0.012

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [37]:
'''
Define lda model using tfidf corpus
'''
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 10, id2word = dictionary, passes = 1, workers=4)

In [38]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.011*"turnbul" + 0.009*"grandstand" + 0.007*"peter" + 0.007*"interview" + 0.007*"malcolm" + 0.006*"august" + 0.005*"islam" + 0.005*"australia" + 0.005*"cricket" + 0.005*"australian"


Topic: 1 Word: 0.010*"drum" + 0.009*"rural" + 0.008*"crash" + 0.007*"stab" + 0.007*"polic" + 0.007*"die" + 0.006*"fatal" + 0.006*"septemb" + 0.006*"novemb" + 0.006*"woman"


Topic: 2 Word: 0.010*"polic" + 0.008*"interview" + 0.008*"driver" + 0.006*"miss" + 0.006*"crash" + 0.006*"queensland" + 0.006*"search" + 0.006*"climat" + 0.005*"charg" + 0.005*"juli"


Topic: 3 Word: 0.017*"countri" + 0.016*"hour" + 0.009*"govern" + 0.009*"podcast" + 0.008*"health" + 0.005*"fund" + 0.005*"octob" + 0.005*"indigen" + 0.005*"monday" + 0.005*"mental"


Topic: 4 Word: 0.010*"live" + 0.007*"leagu" + 0.007*"marriag" + 0.007*"david" + 0.006*"export" + 0.005*"andrew" + 0.005*"world" + 0.005*"australia" + 0.004*"year" + 0.004*"outback"


Topic: 5 Word: 0.009*"coast" + 0.007*"australia" + 0.007*"korea" + 0.007*"n

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [40]:
'''
Text of sample document 4310
'''
print(training_headlines[4310])

rain helps dampen bushfires


In [43]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3404908776283264	 
Topic: 0.025*"govern" + 0.017*"donald" + 0.017*"plan" + 0.016*"chang" + 0.012*"health" + 0.012*"indigen" + 0.011*"council" + 0.011*"work" + 0.011*"fund" + 0.010*"say"

Score: 0.26112931966781616	 
Topic: 0.028*"queensland" + 0.019*"crash" + 0.018*"die" + 0.015*"miss" + 0.014*"peopl" + 0.012*"island" + 0.012*"victoria" + 0.012*"marriag" + 0.011*"countri" + 0.011*"dead"

Score: 0.25837472081184387	 
Topic: 0.024*"perth" + 0.021*"brisban" + 0.014*"show" + 0.014*"polit" + 0.013*"feder" + 0.011*"record" + 0.009*"weather" + 0.008*"time" + 0.008*"year" + 0.007*"farmer"

Score: 0.02000216580927372	 
Topic: 0.050*"polic" + 0.024*"death" + 0.019*"charg" + 0.016*"live" + 0.016*"shoot" + 0.014*"murder" + 0.014*"woman" + 0.012*"fight" + 0.012*"investig" + 0.011*"child"

Score: 0.020001757889986038	 
Topic: 0.025*"north" + 0.022*"canberra" + 0.020*"coast" + 0.016*"say" + 0.015*"labor" + 0.014*"china" + 0.013*"korea" + 0.012*"gold" + 0.012*"senat" + 0.011*"vote"

Score: 0

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [42]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
document_num = 4310
# Our test document is document number 4310
# TODO
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5740954279899597	 
Topic: 0.057*"australia" + 0.025*"world" + 0.018*"open" + 0.016*"final" + 0.013*"win" + 0.012*"leagu" + 0.012*"share" + 0.012*"take" + 0.009*"lead" + 0.009*"young"

Score: 0.2658849358558655	 
Topic: 0.025*"melbourn" + 0.024*"nation" + 0.014*"west" + 0.013*"state" + 0.013*"bank" + 0.013*"street" + 0.012*"forc" + 0.012*"guilti" + 0.011*"game" + 0.011*"parti"

Score: 0.020006675273180008	 
Topic: 0.028*"queensland" + 0.019*"crash" + 0.018*"die" + 0.015*"miss" + 0.014*"peopl" + 0.012*"island" + 0.012*"victoria" + 0.012*"marriag" + 0.011*"countri" + 0.011*"dead"

Score: 0.020002977922558784	 
Topic: 0.052*"trump" + 0.032*"elect" + 0.023*"school" + 0.021*"women" + 0.019*"attack" + 0.017*"tasmanian" + 0.016*"kill" + 0.015*"arrest" + 0.014*"children" + 0.011*"near"

Score: 0.020002558827400208	 
Topic: 0.025*"north" + 0.022*"canberra" + 0.020*"coast" + 0.016*"say" + 0.015*"labor" + 0.014*"china" + 0.013*"korea" + 0.012*"gold" + 0.012*"senat" + 0.011*"vote"

Score:

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [45]:
unseen_document = "cricket is awesome sport"

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

# TODO
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.2750001847743988	 
Topic: 0.025*"melbourn" + 0.024*"nation" + 0.014*"west" + 0.013*"state" + 0.013*"bank" + 0.013*"street" + 0.012*"forc" + 0.012*"guilti" + 0.011*"game" + 0.011*"parti"

Score: 0.2750001847743988	 
Topic: 0.028*"queensland" + 0.019*"crash" + 0.018*"die" + 0.015*"miss" + 0.014*"peopl" + 0.012*"island" + 0.012*"victoria" + 0.012*"marriag" + 0.011*"countri" + 0.011*"dead"

Score: 0.27499961853027344	 
Topic: 0.055*"australian" + 0.023*"test" + 0.015*"break" + 0.014*"week" + 0.011*"public" + 0.010*"hour" + 0.009*"water" + 0.009*"say" + 0.009*"debat" + 0.008*"russia"

Score: 0.02499999664723873	 
Topic: 0.024*"perth" + 0.021*"brisban" + 0.014*"show" + 0.014*"polit" + 0.013*"feder" + 0.011*"record" + 0.009*"weather" + 0.008*"time" + 0.008*"year" + 0.007*"farmer"

Score: 0.02499999664723873	 
Topic: 0.025*"north" + 0.022*"canberra" + 0.020*"coast" + 0.016*"say" + 0.015*"labor" + 0.014*"china" + 0.013*"korea" + 0.012*"gold" + 0.012*"senat" + 0.011*"vote"

Score: 0.02

The model correctly classifies the unseen document with 'x'% probability to the X category.